# Extract GitHub Repository Names

From the potential list of manually mapped CVEs we first extract the GitHub repository names which had potential CVEs

In [169]:
import pandas as pd
import numpy as np
import re
import gc

In [316]:
df = pd.read_csv('./GO CVEs Data - Data.csv')

In [317]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697 entries, 0 to 1696
Data columns (total 9 columns):
CVE ID           1222 non-null object
Package name     1683 non-null object
Ecosystem        1683 non-null object
GH issue         679 non-null object
GH PR            322 non-null object
GH Commit        856 non-null object
Bugzilla         245 non-null object
ML               465 non-null object
Other sources    34 non-null object
dtypes: object(9)
memory usage: 119.4+ KB


In [318]:
df.head()

,CVE ID,Package name,Ecosystem,GH issue,GH PR,GH Commit,Bugzilla,ML,Other sources
0,CVE-2017-9805,struts2-rest-plugin,maven,NaN,NaN,https://github.com/apache/struts/commit/194947...,https://bugzilla.redhat.com/show_bug.cgi?id=14...,NaN,https://blogs.apache.org/foundation/entry/apac...
1,CVE-2018-8617,Microsoft.ChakraCore,nuget,NaN,https://github.com/Microsoft/ChakraCore/pull/5869,NaN,NaN,NaN,NaN
2,CVE-2018-8629,Microsoft.ChakraCore,nuget,NaN,https://github.com/Microsoft/ChakraCore/pull/5869,NaN,NaN,NaN,NaN
3,CVE-2018-6824,Microsoft.ChakraCore,nuget,NaN,https://github.com/Microsoft/ChakraCore/pull/5869,NaN,NaN,NaN,NaN
4,CVE-2018-8618,Microsoft.ChakraCore,nuget,NaN,https://github.com/Microsoft/ChakraCore/pull/5869,NaN,NaN,NaN,NaN


In [319]:
gh_issues = df['GH issue'].dropna().values
print('Before:', gh_issues.shape)
gh_issues = [item.strip().split('\n') for item in gh_issues]
gh_issues = np.array([str(item.strip()) for sublist in gh_issues for item in sublist])
print('After:', gh_issues.shape)

Before: (679,)
After: (720,)


In [320]:
gh_prs = df['GH PR'].dropna().values
print('Before:', gh_prs.shape)
gh_prs = [item.strip().split('\n') for item in gh_prs]
gh_prs = np.array([str(item.strip()) for sublist in gh_prs for item in sublist])
print('After:', gh_prs.shape)

Before: (322,)
After: (357,)


In [321]:
gh_commits = df['GH Commit'].dropna().values
print('Before:', gh_commits.shape)
gh_commits = [item.strip().split('\n') for item in gh_commits]
gh_commits = np.array([str(item.strip()) for sublist in gh_commits for item in sublist])
print('After:', gh_commits.shape)

Before: (856,)
After: (1056,)


In [322]:
gh_links = np.concatenate((gh_issues, gh_commits, gh_prs))
gh_links.shape

(2133,)

In [323]:
gh_links = np.array([item for item in gh_links if 'github' in item])

In [324]:
gh_links

array(['https://github.com/intelliants/subrion/issues/801',
       'https://github.com/ethereum/py-evm/issues/1448',
       'https://github.com/rapid7/metasploit-framework/issues/8064', ...,
       'https://github.com/pallets/werkzeug/pull/1001',
       'https://github.com/boto/boto/pull/3097',
       'https://github.com/jupyter-widgets/ipywidgets/pull/591'],
      dtype='<U179')

In [325]:
pattern = re.compile(r'.*?github.com/(.*?/.*?)/', re.I)
repo_names = np.array(list(filter(None,[pattern.search(item).group(1) if pattern.search(item) else None 
                           for item in gh_links]))) 

In [326]:
repo_names.shape

(1656,)

In [327]:
repo_names = np.unique(repo_names)

In [328]:
repo_names.shape

(507,)

# View total GitHub Events for Repos

In [132]:
import bq_utils as bqu
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

In [133]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'bq_key.json'
gh_archive = bqu.BigQueryHelper(active_project= "githubarchive", 
                                dataset_name = "year")
gh_archive

In [134]:
gh_archive.list_tables()

['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

In [138]:
query = """
SELECT type, count(*)
        FROM `githubarchive.year.*`
        WHERE repo.name in {repos}
        GROUP BY type
""".format(repos=tuple(repo_names))
gh_archive.estimate_query_size(query)

56.20826395787299

In [141]:
df = gh_archive.query_to_pandas(query)
df[df.type.isin(['PushEvent', 'PullRequestEvent', 'IssuesEvent', 
                'PullRequestReviewCommentEvent', 'IssueCommentEvent'])]

,type,f0_
2,IssuesEvent,434016
6,PushEvent,930212
8,PullRequestEvent,746701
13,IssueCommentEvent,2515484
15,PullRequestReviewCommentEvent,769722


# Extract and Save GitHub Issues

In [162]:
query = """
SELECT 
    repo.name as repo_name, 
    type as event_type, 
    actor.id as actor_id,
    actor.login as actor_name,
    JSON_EXTRACT_SCALAR(payload, '$.action') as issue_status,
    JSON_EXTRACT_SCALAR(payload, '$.issue.url') as issue_api_url,
    JSON_EXTRACT_SCALAR(payload, '$.issue.html_url') as issue_url,
    JSON_EXTRACT_SCALAR(payload, '$.issue.user.login') as issue_creator_name,
    JSON_EXTRACT_SCALAR(payload, '$.issue.user.url') as issue_creator_api_url,
    JSON_EXTRACT_SCALAR(payload, '$.issue.user.html_url') as issue_creator_url,
    JSON_EXTRACT_SCALAR(payload, '$.issue.comments') as comment_count,
    JSON_EXTRACT_SCALAR(payload, '$.issue.id') as issue_id,
    JSON_EXTRACT_SCALAR(payload, '$.issue.number') as issue_number,
    JSON_EXTRACT_SCALAR(payload, '$.issue.created_at') as issue_created_at,
    JSON_EXTRACT_SCALAR(payload, '$.issue.updated_at') as issue_updated_at,
    JSON_EXTRACT_SCALAR(payload, '$.issue.closed_at') as issue_closed_at,
    TRIM(REGEXP_REPLACE(
             REGEXP_REPLACE(
                 JSON_EXTRACT_SCALAR(payload, '$.issue.title'), 
                 r'\\r\\n|\\r|\\n', 
                 ' '),
             r'\s{2,}', 
             ' ')) as issue_title,
    TRIM(REGEXP_REPLACE(
             REGEXP_REPLACE(
                 JSON_EXTRACT_SCALAR(payload, '$.issue.body'), 
                 r'\\r\\n|\\r|\\n', 
                 ' '),
             r'\s{2,}', 
             ' ')) as issue_body
        
FROM `githubarchive.year.*`
    WHERE repo.name in {repos}
    AND type = 'IssuesEvent'
    """.replace('{repos}', str(tuple(repo_names)))
gh_archive.estimate_query_size(query)

3279.453662957996

In [163]:
df = gh_archive.query_to_pandas(query)

In [164]:
df.to_csv('GH_issues.csv', index=False)

In [165]:
df.shape

(434016, 18)

In [173]:
del df
gc.collect()

0

# Extract and Save GitHub Commits

In [171]:
query = """
SELECT 
    repo.name as repo_name, 
    type as event_type, 
    actor.id as actor_id,
    actor.login as actor_name,
    JSON_EXTRACT_SCALAR(payload, '$.push_id') as push_id,
    JSON_EXTRACT_SCALAR(payload, '$.size') as size,
    JSON_EXTRACT_SCALAR(payload, '$.distinct_size') as distinct_size,
    JSON_EXTRACT(payload, '$.commits') as commits,
    created_at as rec_created_at
        
FROM `githubarchive.year.*`
    WHERE repo.name in {repos}
    AND type = 'PushEvent'
""".format(repos=tuple(repo_names))
gh_archive.estimate_query_size(query)

3291.882761504501

In [174]:
df = gh_archive.query_to_pandas(query)

In [175]:
df.to_csv('GH_commits_raw.csv', index=False)

In [176]:
df.shape

(930212, 9)

In [178]:
df.head()

,repo_name,event_type,actor_id,actor_name,push_id,size,distinct_size,commits,rec_created_at
0,h2oai/h2o-3,PushEvent,18428990,angela0xdata,2412122536,1,1,"[{""sha"":""5cdb97644ba03998ea78af8222a56036b80f1...",2018-03-18 14:36:23+00:00
1,numpy/numpy,PushEvent,77272,charris,2317952012,2,2,"[{""sha"":""58d47199e2c72b57c05c550567c03fdebbfdf...",2018-02-09 03:05:21+00:00
2,rails/rails,PushEvent,47848,rafaelfranca,2985095948,1,1,"[{""sha"":""8cdb5df7c5c966495730587470295ac2ab3ce...",2018-10-23 22:04:39+00:00
3,apache/lucene-solr,PushEvent,1341245,asfgit,2641477850,1,1,"[{""sha"":""5ae716c412d705570b2dafd423755eb581422...",2018-06-13 11:09:49+00:00
4,apache/hadoop,PushEvent,1341245,asfgit,3037111129,1,1,"[{""sha"":""50ce9fd4e24ce3be863452d40edadd20144c7...",2018-11-10 09:03:35+00:00


In [185]:
new_df = df[~pd.isnull(df['commits'])]

In [186]:
cols = list(new_df.columns)
cols.remove('commits')
new_df['commits'] = new_df['commits'].apply(lambda row: json.loads(row))
new_df = new_df.set_index(cols)['commits'].apply(pd.Series).stack().reset_index()
new_df.columns = cols + ['commit_num', 'commit_details']
commit_details = json_normalize(new_df['commit_details'])
new_df = new_df.drop(columns=['commit_num', 'commit_details'])
new_df = pd.concat([new_df, commit_details], axis=1)

/home/dsarkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [187]:
new_df.shape

(2174563, 14)

In [188]:
new_df.to_csv('GH_commits.csv', index=False)

In [189]:
new_df.head()

,repo_name,event_type,actor_id,actor_name,push_id,size,distinct_size,rec_created_at,author.email,author.name,distinct,message,sha,url
0,h2oai/h2o-3,PushEvent,18428990,angela0xdata,2412122536,1,1,2018-03-18 14:36:23+00:00,7c125b314ee7fec059df0b595622f79463bc36ec@0xdat...,angela0xdata,True,Added update for 3.18.0.4 release,5cdb97644ba03998ea78af8222a56036b80f1efe,https://api.github.com/repos/h2oai/h2o-3/commi...
1,numpy/numpy,PushEvent,77272,charris,2317952012,2,2,2018-02-09 03:05:21+00:00,c63e9d3256de29b0648862e9584cc0360471f3b2@gmail...,Eric Wieser,True,ENH: Show the silenced error and traceback in ...,58d47199e2c72b57c05c550567c03fdebbfdf86d,https://api.github.com/repos/numpy/numpy/commi...
2,numpy/numpy,PushEvent,77272,charris,2317952012,2,2,2018-02-09 03:05:21+00:00,437b12b5887266fca3b5f10a48c6f1b638c2763f@gmail...,Charles Harris,True,Merge pull request #10337 from eric-wieser/war...,4cfac4067a27835de6b0ae204df0e8c91998c1f2,https://api.github.com/repos/numpy/numpy/commi...
3,rails/rails,PushEvent,47848,rafaelfranca,2985095948,1,1,2018-10-23 22:04:39+00:00,84c66fc058fe65d1bb08ca33b21bfe0fa5cb8f17@gmail...,Rafael Mendonça França,True,Do not need to mention the method that is bein...,8cdb5df7c5c966495730587470295ac2ab3ce7fc,https://api.github.com/repos/rails/rails/commi...
4,apache/lucene-solr,PushEvent,1341245,asfgit,2641477850,1,1,2018-06-13 11:09:49+00:00,b7b0c6787377b50e73722d78c6f6b98225fd30c9@apach...,Alan Woodward,True,LUCENE-7444: StandardAnalyzer not longer uses ...,5ae716c412d705570b2dafd423755eb58142212e,https://api.github.com/repos/apache/lucene-sol...


In [193]:
del df
del new_df
gc.collect()

351

# Extract and Save GitHub Pull Requests

In [194]:
query = """
SELECT 
    repo.name as repo_name, 
    type as event_type, 
    actor.id as actor_id,
    actor.login as actor_name,
    JSON_EXTRACT_SCALAR(payload, '$.action') as pr_status,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.id') as pr_id,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.number') as pr_number,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.url') as pr_api_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.html_url') as pr_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.diff_url') as pr_diff_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.patch_url') as pr_patch_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.user.login') as pr_creator_name,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.user.url') as pr_creator_api_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.user.html_url') as pr_creator_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.created_at') as pr_created_at,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.updated_at') as pr_updated_at,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.closed_at') as pr_closed_at,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.merged_at') as pr_merged_at,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.merged') as pr_merged_status,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.comments') as pr_comments_count,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.review_comments') as pr_review_comments_count,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.commits') as pr_commits_count,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.additions') as pr_additions_count,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.deletions') as pr_deletions_count,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.changed_files') as pr_changed_files_count,    
    TRIM(REGEXP_REPLACE(
             REGEXP_REPLACE(
                 JSON_EXTRACT_SCALAR(payload, '$.pull_request.title'), 
                 r'\\r\\n|\\r|\\n', 
                 ' '),
             r'\s{2,}', 
             ' ')) as pr_title,
    TRIM(REGEXP_REPLACE(
             REGEXP_REPLACE(
                 JSON_EXTRACT_SCALAR(payload, '$.pull_request.body'), 
                 r'\\r\\n|\\r|\\n', 
                 ' '),
             r'\s{2,}', 
             ' ')) as pr_body
        
FROM `githubarchive.year.*`
    WHERE repo.name in {repos}
    AND type = 'PullRequestEvent'
""".replace('{repos}', str(tuple(repo_names)))
gh_archive.estimate_query_size(query)

3279.453662957996

In [196]:
df = gh_archive.query_to_pandas(query)

In [197]:
df.shape

(746701, 27)

In [198]:
df.to_csv('GH_pull_requests.csv', index=False)

# Save list of Repo Names (for tracking)

In [314]:
repos = pd.DataFrame(repo_names, columns=['repo_names'])
repos.to_csv('GH_repo_names.csv', index=False)

In [315]:
repos.shape

(507, 1)